In [3]:
%load_ext autoreload
%autoreload 2

In [52]:
import collections
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import scipy.spatial

import voxart

In [10]:
import cProfile
import pstats
from pstats import SortKey

# 0.profile

In [12]:
goal = voxart.Goal.from_image(Image.open("../assets/chiral_7.png"))

In [13]:
opts = voxart.SearchOptions()
opts.filled_num_batches = 2
cProfile.run('results = voxart.search(goal, opts)', '0.profile')

SearchOptions(filled_batch_size=3, filled_num_batches=2, filled_strategy='random_clear_front', filled_num_to_pursue=20, connector_num_iterations_per=30, connector_frac=0.4, top_n=20, obj_func=None)


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [15]:
p = pstats.Stats('0.profile')

In [16]:
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()

Thu Mar 23 19:36:46 2023    profile0

         117476310 function calls (117398762 primitive calls) in 100.559 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      9/1    0.000    0.000  100.573  100.573 {built-in method builtins.exec}
        1    0.000    0.000  100.573  100.573 <string>:1(<module>)
        1    0.015    0.015  100.573  100.573 search.py:558(search)
       40    0.287    0.007   58.386    1.460 search.py:360(search_connectors)
     7200   15.017    0.002   57.818    0.008 search.py:302(get_shortest_path_to_targets)
     9392    0.033    0.000   41.376    0.004 search.py:160(add)
     9392    0.219    0.000   41.318    0.004 search.py:107(__call__)
     7680    1.408    0.000   40.974    0.005 search.py:493(count_unsupported)
   620736   17.740    0.000   39.269    0.000 search.py:479(is_vox_unsupported)
  8682249   10.020    0.000   37.668    0.000 search.py:282(get_neighbors)
8499463/8498951    4.148 

In [17]:
p.strip_dirs().sort_stats(SortKey.TIME).print_stats()

Thu Mar 23 19:36:46 2023    profile0

         117476310 function calls (117398762 primitive calls) in 100.559 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   620736   17.740    0.000   39.269    0.000 search.py:479(is_vox_unsupported)
  6930261   16.316    0.000   16.316    0.000 {built-in method numpy.array}
     7200   15.017    0.002   57.818    0.008 search.py:302(get_shortest_path_to_targets)
  8682249   10.020    0.000   37.668    0.000 search.py:282(get_neighbors)
 30129674    6.633    0.000    6.633    0.000 <string>:2(__lt__)
  1583338    4.590    0.000    4.590    0.000 {method 'reduce' of 'numpy.ufunc' objects}
8499463/8498951    4.148    0.000   31.472    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
  6930261    4.002    0.000   26.157    0.000 <__array_function__ internals>:177(copy)
  2808440    3.180    0.000    7.603    0.000 {built-in method _heapq.heappop}
  5351441    

# get_neighbors

get_neighbors is suprisingly slow

Let's investigate

In [19]:
def get_neighbors_orig(vox, size):
    vox = np.asarray(vox)
    if vox.shape != (3,):
        raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    for axis, delta in itertools.product([0, 1, 2], [-1, 1]):
        newval = vox[axis] + delta
        if newval < 0 or newval >= size:
            continue
        neighbor = np.copy(vox)
        neighbor[axis] = newval
        yield neighbor


In [53]:
%timeit collections.deque(get_neighbors_orig(np.array([1, 2, 3]), 4), maxlen=0)

8.06 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [26]:
def get_neighbors_1(vox, size):
    #vox = np.asarray(vox)
    #if vox.shape != (3,):
    #    raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    for axis, delta in itertools.product([0, 1, 2], [-1, 1]):
        newval = vox[axis] + delta
        if newval < 0 or newval >= size:
            continue
        neighbor = np.copy(vox)
        neighbor[axis] = newval
        yield neighbor


In [54]:
%timeit collections.deque(get_neighbors_1(np.array([1, 2, 3]), 4), maxlen=0)

7.81 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [28]:
DELTAS = list(itertools.product([0, 1, 2], [-1, 1]))
def get_neighbors_2(vox, size):
    #vox = np.asarray(vox)
    #if vox.shape != (3,):
    #    raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    for axis, delta in DELTAS:
        newval = vox[axis] + delta
        if newval < 0 or newval >= size:
            continue
        neighbor = np.copy(vox)
        neighbor[axis] = newval
        yield neighbor

In [55]:
%timeit collections.deque(get_neighbors_2(np.array([1, 2, 3]), 4), maxlen=0)

7.49 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [84]:
def get_neighbors_3(vox, size):
    vox = np.array(vox)
    if vox.shape != (3,):
        raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    for axis, delta in itertools.product([0, 1, 2], [-1, 1]):
        newval = vox[axis] + delta
        if newval < 0 or newval >= size:
            continue
        #neighbor = np.copy(vox)
        vox[axis] = newval
        yield vox
        vox[axis] -= delta


In [85]:
list(get_neighbors_3(np.array([1, 2, 3]), 4))

[array([1, 2, 3]),
 array([1, 2, 3]),
 array([1, 2, 3]),
 array([1, 2, 3]),
 array([1, 2, 3])]

In [86]:
np.fromiter(get_neighbors_3(np.array([1, 2, 3]), 4), dtype=(int, 3))

array([[0, 2, 3],
       [2, 2, 3],
       [1, 1, 3],
       [1, 3, 3],
       [1, 2, 2]])

In [87]:
orig_vox = np.array([1, 2, 3])
for neigh in get_neighbors_3(orig_vox, 4):
    print(orig_vox, neigh)

[1 2 3] [0 2 3]
[1 2 3] [2 2 3]
[1 2 3] [1 1 3]
[1 2 3] [1 3 3]
[1 2 3] [1 2 2]


In [88]:
%timeit collections.deque(get_neighbors_3(np.array([1, 2, 3]), 4), maxlen=0)

4.27 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [70]:
DELTA_MATRIX = np.array([
    [-1, 0, 0],
    [1, 0, 0],
    [0, -1, 0],
    [0, 1, 0],
    [0, 0, -1],
    [0, 0, 1],
    ])
def get_neighbors_4(vox, size):
    all_neighs = vox + DELTA_MATRIX
    #display(all_neighs)
    #vox = np.asarray(vox)
    #if vox.shape != (3,):
    #    raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    invalid = np.logical_or.reduce((all_neighs < 0) | (all_neighs >= size), axis=1)
    #display(invalid)
    for neigh_idx in range(6):
        #print(invalid[neigh_idx])
        if invalid[neigh_idx]:
            continue
        yield all_neighs[neigh_idx]

In [71]:
list(get_neighbors_4(np.array([1, 2, 3]), 4))

[array([0, 2, 3]),
 array([2, 2, 3]),
 array([1, 1, 3]),
 array([1, 3, 3]),
 array([1, 2, 2])]

In [72]:
%timeit collections.deque(list(get_neighbors_4(np.array([1, 2, 3]), 4)), maxlen=0)

9.01 µs ± 79.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [62]:
DELTA_MATRIX = np.array([
    [-1, 0, 0],
    [1, 0, 0],
    [0, -1, 0],
    [0, 1, 0],
    [0, 0, -1],
    [0, 0, 1],
    ])
def get_neighbors_5(vox, size):
    all_neighs = vox + DELTA_MATRIX
    #display(all_neighs)
    #vox = np.asarray(vox)
    #if vox.shape != (3,):
    #    raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    #invalid = np.logical_or.reduce((all_neighs < 0) | (all_neighs >= size), axis=1)
    #display(invalid)
    for neigh_idx in range(6):
        #print(invalid[neigh_idx])
        #if invalid[neigh_idx]:
        #    continue
        yield all_neighs[neigh_idx, :]

In [63]:
%timeit collections.deque(list(get_neighbors_5(np.array([1, 2, 3]), 4)), maxlen=0)

4.78 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [73]:
DELTA_MATRIX = np.array([
    [-1, 0, 0],
    [1, 0, 0],
    [0, -1, 0],
    [0, 1, 0],
    [0, 0, -1],
    [0, 0, 1],
    ])
def get_neighbors_6(vox, size):
    all_neighs = vox + DELTA_MATRIX
    #display(all_neighs)
    #vox = np.asarray(vox)
    #if vox.shape != (3,):
    #    raise ValueError(f"Only suport 3D neightbors, got shape {vox.shape}")
    #invalid = np.logical_or.reduce((all_neighs < 0) | (all_neighs >= size), axis=1)
    #display(invalid)
    for neigh_idx in range(6):
        neigh = all_neighs[neigh_idx]
        if np.any(neigh < 0) or np.any(neigh >= size):
            continue
        #print(invalid[neigh_idx])
        #if invalid[neigh_idx]:
        #    continue
        yield neigh

In [74]:
%timeit collections.deque(list(get_neighbors_6(np.array([1, 2, 3]), 4)), maxlen=0)

71.5 µs ± 8.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Based on all this, I am switching to the form in get_neighbors_3

# 1.profile

In [89]:
goal = voxart.Goal.from_image(Image.open("../assets/chiral_7.png"))

In [90]:
opts = voxart.SearchOptions()
opts.filled_num_batches = 2
cProfile.run('results = voxart.search(goal, opts)', '1.profile')

SearchOptions(filled_batch_size=3, filled_num_batches=2, filled_strategy='random_clear_front', filled_num_to_pursue=20, connector_num_iterations_per=30, connector_frac=0.4, top_n=20, obj_func=None)


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [91]:
p = pstats.Stats('1.profile')

In [92]:
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()

Thu Mar 23 21:20:48 2023    1.profile

         83073659 function calls (82996180 primitive calls) in 66.898 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   66.913   66.913 {built-in method builtins.exec}
        1    0.001    0.001   66.913   66.913 <string>:1(<module>)
        1    0.019    0.019   66.913   66.913 search.py:566(search)
       40    0.318    0.008   33.141    0.829 search.py:368(search_connectors)
     9392    0.037    0.000   32.983    0.004 search.py:160(add)
     9392    0.228    0.000   32.919    0.004 search.py:107(__call__)
     7680    1.446    0.000   32.563    0.004 search.py:501(count_unsupported)
     7200   12.479    0.002   32.519    0.005 search.py:310(get_shortest_path_to_targets)
   620608   16.322    0.000   30.812    0.000 search.py:487(is_vox_unsupported)
      480    0.021    0.000   16.758    0.035 search.py:520(search_bottom_location)
1569012/1568500   

In [93]:
p.strip_dirs().sort_stats(SortKey.TIME).print_stats()

Thu Mar 23 21:20:48 2023    1.profile

         83073659 function calls (82996180 primitive calls) in 66.898 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   620608   16.322    0.000   30.812    0.000 search.py:487(is_vox_unsupported)
     7200   12.479    0.002   32.519    0.005 search.py:310(get_shortest_path_to_targets)
  8710547    7.197    0.000    8.488    0.000 search.py:282(get_neighbors)
 30262325    6.556    0.000    6.556    0.000 <string>:2(__lt__)
  1582892    4.390    0.000    4.390    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  2818768    3.110    0.000    7.603    0.000 {built-in method _heapq.heappop}
  1554716    1.929    0.000    6.996    0.000 fromnumeric.py:69(_wrapreduction)
  5376709    1.877    0.000    3.944    0.000 {built-in method _heapq.heappush}
  5378809    1.760    0.000    1.760    0.000 <string>:2(__init__)
     7680    1.446    0.000   32.563    0.004 search.py:501(count_unsupport

# is_vox_unsupported

In [99]:
from typing import Tuple

In [105]:
_, design = results.best()[0]
design.bottom_location

array([1, 0, 1], dtype=int32)

In [103]:
def is_vox_unsupported_orig(design: voxart.Design, vox: Tuple[int, int, int]):
    if np.all(vox == design.bottom_location * (design.size - 1)):
        return False
    for neigh_vox in voxart.get_neighbors(vox, design.size):
        if design.voxels[tuple(neigh_vox)] == voxart.EMPTY:
            continue
        diff = neigh_vox - vox
        # print(f"For {vox}, bottom {design.bottom_location}, neighbor {neigh_vox}, diff {diff}, "
        #       f"{design.bottom_location * 2 - 1} {diff == (design.bottom_location * 2 - 1)}")
        if np.sum(diff == (design.bottom_location * 2 - 1)) == 1:
            return False
    return True

In [104]:
%timeit is_vox_unsupported_orig(design, (3, 3, 3))

43.3 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [118]:
def is_vox_unsupported_0(design: voxart.Design, vox: Tuple[int, int, int]):
    if vox == tuple(design.bottom_location * (design.size - 1)):
        return False
    for neigh_vox in voxart.get_neighbors(vox, design.size):
        if design.voxels[tuple(neigh_vox)] == voxart.EMPTY:
            continue
        diff = neigh_vox - vox
        # print(f"For {vox}, bottom {design.bottom_location}, neighbor {neigh_vox}, diff {diff}, "
        #       f"{design.bottom_location * 2 - 1} {diff == (design.bottom_location * 2 - 1)}")
        if np.sum(diff == (design.bottom_location * 2 - 1)) == 1:
            return False
    return True

In [119]:
%timeit is_vox_unsupported_0(design, (3, 3, 3))

38.6 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [137]:
# Note that this is pretty hacky because I'm doing some work outside of the timed function
# but I'm imaginging I woudl do it in count_unsupported
top = ((design.bottom_location + 1) % 2) * (design.size - 1)
print(top)
support_diffs = list(n - top for n in voxart.get_neighbors(top, design.size))
print(support_diffs)
def is_vox_unsupported_1(design: voxart.Design, vox: Tuple[int, int, int]):
    if vox == tuple(design.bottom_location * (design.size - 1)):
        return False
    for neigh_diff in support_diffs:
        #print(neigh_diff)
        neigh_vox = vox + neigh_diff
        #print(neigh_vox)
        if -1 in neigh_vox or design.size in neigh_vox:
            continue
        if design.voxels[tuple(neigh_vox)] != voxart.EMPTY:
            return False
    return True

[0 6 0]
[array([1, 0, 0], dtype=int32), array([ 0, -1,  0], dtype=int32), array([0, 0, 1], dtype=int32)]


In [138]:
%timeit is_vox_unsupported_1(design, (3, 3, 3))

35.5 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [141]:
def is_vox_unsupported_2(design: voxart.Design, vox: Tuple[int, int, int]):
    if vox == tuple(design.bottom_location * (design.size - 1)):
        return False

    diff = design.bottom_location[0] * 2 - 1
    neigh_vox = (vox[0] + diff, vox[1], vox[2])
    if neigh_vox[0] != -1 and neigh_vox[0] != design.size:
        if design.voxels[neigh_vox] != voxart.EMPTY:
            return False

    diff = design.bottom_location[1] * 2 - 1
    neigh_vox = (vox[0], vox[1] + diff, vox[2])
    if neigh_vox[1] != -1 and neigh_vox[1] != design.size:
        if design.voxels[neigh_vox] != voxart.EMPTY:
            return False

    diff = design.bottom_location[2] * 2 - 1
    neigh_vox = (vox[0], vox[1], vox[2] + diff)
    if neigh_vox[2] != -1 and neigh_vox[2] != design.size:
        if design.voxels[neigh_vox] != voxart.EMPTY:
            return False

    return True

In [142]:
%timeit is_vox_unsupported_2(design, (3, 3, 3))

10.2 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# 2.profile

In [143]:
goal = voxart.Goal.from_image(Image.open("../assets/chiral_7.png"))

In [144]:
opts = voxart.SearchOptions()
opts.filled_num_batches = 2
cProfile.run('results = voxart.search(goal, opts)', '2.profile')

SearchOptions(filled_batch_size=3, filled_num_batches=2, filled_strategy='random_clear_front', filled_num_to_pursue=20, connector_num_iterations_per=30, connector_frac=0.4, top_n=20, obj_func=None)


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [145]:
p = pstats.Stats('2.profile')

In [146]:
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()

Thu Mar 23 22:31:39 2023    2.profile

         61576570 function calls (61499091 primitive calls) in 39.109 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   40.862   40.862 {built-in method builtins.exec}
        1    0.000    0.000   40.862   40.862 <string>:1(<module>)
        1    0.013    0.013   40.862   40.862 search.py:581(search)
       40    0.304    0.008   32.287    0.807 search.py:368(search_connectors)
     7200   12.191    0.002   31.689    0.004 search.py:310(get_shortest_path_to_targets)
     9392    0.030    0.000    7.813    0.001 search.py:160(add)
     9392    0.196    0.000    7.762    0.001 search.py:107(__call__)
     7680    1.071    0.000    7.444    0.001 search.py:516(count_unsupported)
  2807350    3.025    0.000    7.400    0.000 {built-in method _heapq.heappop}
 30124875    6.392    0.000    6.392    0.000 <string>:2(__lt__)
   620656    5.552    0.000    6.139  

In [147]:
p.strip_dirs().sort_stats(SortKey.TIME).print_stats()

Thu Mar 23 22:31:39 2023    2.profile

         61576570 function calls (61499091 primitive calls) in 39.109 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     7200   12.191    0.002   31.689    0.004 search.py:310(get_shortest_path_to_targets)
 30124875    6.392    0.000    6.392    0.000 <string>:2(__lt__)
   620656    5.552    0.000    6.139    0.000 search.py:487(is_vox_unsupported)
  6488056    4.849    0.000    5.595    0.000 search.py:282(get_neighbors)
  2807350    3.025    0.000    7.400    0.000 {built-in method _heapq.heappop}
  5353399    1.821    0.000    3.842    0.000 {built-in method _heapq.heappush}
     7680    1.071    0.000    7.444    0.001 search.py:516(count_unsupported)
  8988671    0.927    0.000    0.927    0.000 __init__.py:118(voxels)
  1140013    0.748    0.000    0.748    0.000 {built-in method numpy.array}
  2840259    0.568    0.000    0.568    0.000 __init__.py:114(size)
       40    0.304